# Modeling Training Notebook
<hr>

This is the Jupyter Notebook to get you started on writing a simple Bayesian
Model. To complete this, you need basic understanding of programming in python
and of Bayesian inference. If you run into problems, you should first try do you
research before asking me for solutions. 

## Task Overview
The task for your model is identitcal to the task we've given to the subjects.
That is, given a series of combinations of objects, what is the most probable deterministic
rule that govern the outcomes (explosion or no explosion) of all these
combinations? 

There are several key details you should know before starting. First, there are
three binary features: 
$$
\begin{array}{rcl}
\text{Color} &=& \{\text{Red, Blue}\} \\
\text{Shape} &=& \{\text{Circle, Square}\} \\
\text{Size}  &=& \{\text{Large, Small}\} \\
\end{array}
$$
This yields $2^3 = 8$ unique objects. Next, these objects are arranged in a
2-object *multiset combination*, that is to say, objects can be repeated in this
combination. As a result, there are $nCr(8+2-1, 2) = 36$ unique multiset
combinations (if you are curious, this is calculated through *multinomial
coefficients*, which is equivalent to dipositing $8$ distinct objects into $2$
distinct bins).

A set of $36$ 2-object combination seems small, but the rule learning problem that stems
from this set is immense. To give you a sense of the complexity at hand, we can
consider the number of possible *extensions* of this rule. Since we are working
with boolean outcome (a combination will either trigger the effect or not), the
extension of a rule is a partition of the $36$ item into two *complementary*
sets. Allowing empty set, this will give us $2^{36} = 6.87194767\times10^{10}$
unique partitions (and hence unique rules). To be clear, this is an extremely large hypothesis space in
the context of higher cognition, and human learners probably only test a tiny
fraction of these rules. It is our job to figure what what are the rules that
makes up this small fraction, and why they decide on these rules instead of
other.

A common way to approach this question is to investigate these rules from the
*intension* perspective. For example, we can represent the rules as first-order
logic expressions. Instead of investigating all the $6.87194767\times10^{10}$ of
unique rules, we can instead focus on several *families* of rules that have
specific forms with the assumption that only these families are readily
avaliable to the learners. In this task, you will implement two of the rule families
we've used. The *type-singular* family specifies the feature combinations within an
obect boundary; the *counting conjunction* family specifies the overall frequency of
features within the 2-object combination regardless of the object boundary. We will get into the details of these rules later.

## Model Architecture
The model is split into two parts: object/rule representation and bayesian modeling. The
representation part deals with representing the stimuli, generating rules
that fill out a hypotheses space, and verifying whether a given stimuli obeys a
given rule. The bayesian part manages the probablistic distribution (*posterior*)
over all rules in the hypotheses space, and updates this distribution whenever a
new stimuli is encountered. You will first implement the rule representation
part before writing out the bayesian model.

Some advices before you jump in: 
1. Think carefully about how you represent the rules and the stimuli. This will
   be the core part of your model and will be the hardest one to change.
2. When you implement a model, you should always keep in mind that you might
   need to modify your model in the future (e.g., you need to accomadate
   different rules or need a more advanced bayesian model). Don't engineer
   yourself into a corner.
3. Avoid premature optimization. This might not be applicable to you, but I
   usually get distracted by trying to write the most efficient algorithm for a
   small task at the expense of modularity and readability. You should only
   optimize your code if some operations eat up a huge chunk of your runtime.

In [1]:
import numpy as np
import random
from itertools import product, combinations, permutations

Above are some basic libraries you will need to get your model to work, but feel
free to import more. Note, however, you should not import readily made models for
bayesian modeling.

<hr>

# Section 1. Internal Representation
The core of your model is the representations (in terms of data structure) it
use. There are two important parts: object representations encode the stimuli,
and the rule representations encode the rules you give to your model. Keep in
mind that the two representations must be able to recognize each other. That is,
you must design these two systems of representation in a way that the rules must
be able to determine whether a given stimulus satisfy itself.

## 1a. Object Representation
Recall that we have $8$ unique objects that varies in $3$ binary dimensions.
Each training stimuli comes in with $2$ objects, and each generalization stimuli
come with $3$. 

Along with this notebook there is also the `all_data.xslx` file, which encode
all the 211 stimuli a subject have seen. You will find the detailed information
in the excel file, but the object coding is shown below:
$$
\begin{array}{rcl}
2 &:& \text{Red, Circle, Large} \\
3 &:& \text{Red, Circle, Small} \\
5 &:& \text{Red, Triangle, Large} \\
7 &:& \text{Red, Triangle, Small} \\
11 &:& \text{Blue, Circle, Large} \\
13 &:& \text{Blue, Circle, Small} \\
17 &:& \text{Blue, Triangle, Large} \\
19 &:& \text{Blue, Triangle, Small} \\
\end{array}
$$
Each stimulus comes in as a sequence. For example, a sequence of $[2,3]$ means
the stimulus is a red large circle on the left and a red small circle on the
right. As a side note, you might notice that the object codings are all in prime number, and
this is intentional. In my implementation, I represented object with
prime number as a simplifed form of Godel encoding, since doing so will enforce
the products of every sets of objects to be unique. Checking whether two
sequences are the same combinations boils down to checking if they have the same
product. This sounds like a clever design, but it's practically inferior (unless
you use Godel numbering to code individual features and somehow managed to
exploit those prime numbers in rule verification) since it's slow (not in
checking products, but in situations when you need to do prime factorization) and unreadible
without a table. I advice you to come up with your own way of representing
objects, but feel free to exploit Godel encoding if you so desire.

You have two specific tasks:
1. **You should device a representation scheme for the stimuli that encodes the
   three features of each object and preserve the order the objects in a
   stimulus.** Although the rule we test do not care about this order, we still
   need to keep this information in case we want to take it into account in the
   future.
2. **You should write some code to read the subject's data in the excel file and
  recast all the stimuli in your own object representation**. You don't need to
  read the excel file directly; you can, for example, just copy the subjects' data
  into a txt file or the csv file and read them with numpy or pandas.

In [ ]:
# Your code here

## 1b. Type-Singular Rule Representation
Now that you have created a representation scheme for objects, you can move to
create a representation schemes for the rules. The rule space you will implement
is the object-oriented rule space. Generally speaking, an object-oriented rule
take the form of the following:
$$
\exists_x[\text{Color(x) = Red} \land \text{Shape(x) = Circle} \land
\text{Size(x) = Large}]
$$
Intuitively, this says: "*there is a Large Red Circle*". Here are some critical constraints:
1. In each object clause, there can be at least 1 feature specification at most 3, and no
   two specifications describe the same feature dimension. For example, you
   cannot have $\exists_x[\text{Color(x) = Red} \land \text{Color(x) = Blue}]$.
2. In each object clause, all feature specifications are connected through
   conjunctions
3. In each rule, there is exactly 1 object clause.

Given this specification, there are $26$ different rules. Make sure you
understand why they add up to $26$. To give you a hint, you should approach this
problem by dividing the rules into three categories: rules with $1$, $2$, and
$3$ feature specifications.  


To help you with this task, I include a helper function to generate
combinations. You might need this to generate the rule families.

In [2]:
# Generate sets of size r from sample space of n elements
def multiset_comb(n, r, constructor = tuple):
	indices = [0] * r

	yield constructor(indices)
	while True:
		# find the right-most index that does not reach the end
		for i in reversed(range(r)):
			if indices[i] != n - 1:
				break
		else:
			# if all indices are n - 1, done
			return
		# e.g. if n = 3, (0, 1, 2) --> (0, 2, 2)
		indices[i:] = [indices[i] + 1] * (r - i)
		yield constructor(indices)

The multiset_comb function generate *indexes* that you can use. The nature of
this function should become clear to you with the following demonstration.
Suppose that we want to generate all Type-Singular rules that have only one
feature specification. To start, we know that each rule chooses one feature from
the three. Using $n = 3$ and $r = 1$ we can generate the feature position indices:

In [3]:
Feature_Positions = list(multiset_comb(3,1))
for pos in Feature_Positions: print(pos)

(0,)
(1,)
(2,)


As you can see, the `multiset_comb` function give us a sequence of indices to
select. When we iterate through this sequence, we select the first, second, and
third feature. 

You might notice that the output of `multiset_comb` is converted to a list. This
is because the function gives a *generator* that does not store anything but
generates results on the go. One certain advantage of generators over lists is
that they take minimal memory. However, this means you cannot index a generator
like you index a list, and you need to reset the generator once you finish a
loop if you want to reuse it. Generators are useful when we need to generate a large
group of things but only need to use a fraction of them. In our project this is
not a big deal. You can read more about generators in Python's official
documentation.

We can map these indexes to select these features: 

In [4]:
Feature_Names = ["Color", "Shape", "Size"]
for pos in Feature_Positions:
    feature_index = pos[0]
    print("Current Feature Selection:", Feature_Names[feature_index])

Current Feature Selection: Color
Current Feature Selection: Shape
Current Feature Selection: Size


Next, we know that each feature has two states. When a feature is selected,
there are two states this feature specification can take. We will use
`multiset_comb` in a similar manner to generate the two states.

In [5]:
Features = [["Red", "Blue"], ["Circle", "Square"], ["Large", "Small"]]
Feature_States = list(multiset_comb(2,1))
for st in Feature_States:
    state_index = st[0]
    print("Current Feature State for Color:", Features[0][state_index])

Current Feature State for Color: Red
Current Feature State for Color: Blue


Now we need to combine the two lists together. This is relatively simple,
because the feature states and feature positions are independent. This means we
can just get the Cartesian product of the two list to get all possible combination of
positions and states:

In [6]:
All_Rules = list(product(Feature_Positions, Feature_States))
for rule in All_Rules:
    print(rule)

((0,), (0,))
((0,), (1,))
((1,), (0,))
((1,), (1,))
((2,), (0,))
((2,), (1,))


Each iteration give us a pair of indices, with the first and second indicating
the position and state of the features. We can them map them to the actual
features to form rules:

In [8]:
for rule in All_Rules:
    pos = rule[0][0]
    st = rule[1][0]
    print(Feature_Names[pos],":",Features[pos][st])

Color : Red
Color : Blue
Shape : Circle
Shape : Square
Size : Large
Size : Small


This will give you all $6$ Type-Singular rules that have only $1$ feature
specification. You can generalize this process to rules that have $2$ and $3$
feature specifications. For example, when you use `multiset_comb` to generate
the position of $2$ features, you will get a sequence of pairs of indices:

In [9]:
Two_Specs_Pos = list(multiset_comb(3,2))
for pos_pair in Two_Specs_Pos: print(pos_pair)

(0, 0)
(0, 1)
(0, 2)
(1, 1)
(1, 2)
(2, 2)


You will need to figure out a way to apply this same process for the rules with
2 and 3 feature specifications. Specifically, your tasks are:
1. **Develop a way to represent the rules.** Keep in mind that your rule
   representation should be made in a way that it is easy to verify whether a
   given object combination satisfy the rule (you should consider situations
   where 3 objects show up together to account for the generalization stimuli). Also, you should keep your rule representation
   to be uniform, so it's a good idea to read the next section about counting
   conjunctions before deciding on a uniform representation of the rule
   famileis.
2. **Generate all the $26$ rules in the Type-Singular rule family**.
3. **Select $3$ representative rules from this family and verify that they accept
   and reject all $36$ stimuli correctly.**

In [ ]:
# Your code here

## 1c. Counting Conjuntion Rule Representation
Generally speaking, a counting conjunction rule takes the following form:
$$
\exists_o^{=2}[\text{Color(o) = Red}] \land \exists_o^{=1}[\text{Shape(o) =
Circle}] \land \exists_o^{=0}[\text{Size(o) = Large}]
$$
Intuitively, this says *there are two red things, exactly one circle, and no large thing*. Note that the counting conjunction rules *do not* care
about the distribution of these features within the two objects. As long as the
features of the two object added together satisfy the specifications, the
stimulus satisfy the rule. Again, here are some constraints:
1. There is at least $1$ feature specification and at most $6$.
2. Each feature specification can count to $0$, $1$, and $2$
3. Exclude the rules that has no positive extensions (e.g., rules that asks for
   $2$ large things and $2$ small things)

Note that the counting conjunctions only consider *exact* matches. When we only
have two objects and when each feature only has binary states, a lot of these
rules will be describing the same extension. For example, the rule that asks for
two red things is the same as the rule that asks for no blue thing. For now, we
will allow these repeatitions. 

This problem is easier to solve if we disregard **constraint
3** and consider the other constraints first. One way to approach this
problem is to extend our methods in **1b**, where we divide the rule families
into three categories: rules that specify $1$, $2$, and $3$ features. Within the
$1$ specification family, there are $45$ rules. This is because each
state of the feature can have $4$ counting quantifications:
$$
[+0, =0, =1, =2]
$$
where $+0$ here encodes the quantification that does not care about this
feature state. For example, the rule $\exists o^{+0}[\text{Color(o) = Red}]
\land \exists o^{=2}[\text{Color(o) = Blue}]$ only asks for $2$ blue things.
$+0$ is not a standard notation and you can make this into whatever notation you
want. We only include this here to make rule generation easier. Now, since each feature state has $4$ quantifications, each feature will have
$4^2$ unique configurations, and $4^2 - 1 = 15$ configurations that are
non-empty. As a result, the $1$ specification sub-family has $\text{nCr}(3,1) \times
15 = 45$ rules. Following the same principle, the $2$ specification sub-family
has $\text{nCr}(3,2) \times 15^2 = 675$ rules, and the $3$ sub-family has
$\text{nCr}(3,3) \times 15^3 = 3375$ rules.  

You can follow this method outlined above. However, there is also a much simpler
way to solve this problem (hint: $45 + 675 + 3375 = 4095$). Free free to try
this method instead.

Once we have all the possible rules in the family, we can now consider
**constraint 3**. Here we can take the easy way out through brute-force. That
is, we simply check all the $4095$ rules with the $36$ stimuli, and we cast out
the rules satisified by none of the stimuli. This will give us $999$ workable
rules in the family.

Hence, your tasks in this section are:
1. **Generate all $4095$ rules**. You are free to choose your method.
2. **Cast out impossible rules to obtain the $999$ rules**.
3.  **Select $3$ representative rules from this family and verify that they accept
   and reject all $36$ stimuli correctly.**

In [ ]:
# Your code here

<hr>

# Section 2. Bayesian Modeling

## 2a. Prior
Now that we have set up the object and rule representation, we have all the
required machinary to tackle the Bayesian model. Given a rule $h$ (hypothesis)
in a family $\mathcal{H}$ (hypothesis space), the uniform prior function is given
by:
$$
p(h) = \frac{1}{|\mathcal{H}|}
$$
This is an uninformative prior. Typical Bayesian models usually employ a complex
prior, either penalizing the length of the rule or defining exact probablity of
production rules in a Context-Free Grammar that are used to generate the rules.
However, the uniform prior is good enough for our purpose here. 

However, one trick your should learn to master is to cast these probabilities in *log*
space. The reason behind this maneuver has to do with *floating point error*.
The short story is that computers operate on discrete representations of the
continuous number space, and these imprecisions add up with lengthy
computations. Computation of probability is especially susceptible to this error
since we are dealing with very small numbers. Casting probability computation
into log space circumvent this problem to some degree and offer good enough
precision.

The basic idea you need to know is that, in log space, multiplications and
divisions become addition and substraction. Beside `np.log` and `np.exp`, you
probably need the `logsumexp` function that calculate the log of the sum of the
exponenets of the input. This function is imported below:


In [ ]:
from scipy.special import logsumexp

Your tasks are:
1. **Write a function to generate prior probability for the two rule families in
   log space**.
2. **Check that the log prior sums to 0**. You should do this by applying
   `logsumexp` to your prior probabilities.

In [ ]:
# Your code here

## 2b. Likelihood
Given a stimuli at trial i $s_i$ and its outcome $\mathcal{O}(s_i)$, we can obtain its
likelihood conditioned on a specific rule through the following formula:
$$
p(s_i, \mathcal{O}(s_i)|h \in \mathcal{H}) = 
    \begin{cases}
    1 - m  & h \textrm{ matches } s_i, \mathcal{O}(s_i) \\
    m & \textrm{otherwise} \\
    \end{cases}
$$
where $0 < m < 0.5$ is the mismatch free parameter. You can understand $m$ as the
uncertainty of the model: the higher $m$ is, the less certain is the model in
the observation. Higher $m$ results in slower convergence, which means the model
will take more trials to infer the ground truth. Here we will just fix $m$ to be
$0.1$, but it is a good idea to make this a variable that you can change in the
future (for **section 4**). Note that "$h$ matches $s_i,
\mathcal{O}(s_i)$" is the same as $s_i$ satisfying the rule, which you have
implemented in **section 1**.

Your task in this section is:
1. **Implement the likelihood function for both families**. Remember to cast
   these likelihoods into the log space. 

In [ ]:
# Your code here

## 2c. Marginal/Prediction
The model makes a prediction about the outcome of a particular stimulus $s_i$ by
generating the probability that the stimulus triggers the effect $p(\mathcal{O}(s_i)
= 1)$. To do this, we take the product of likelihood and prior of individual
rules and sum them together:
$$
p(\mathcal{O}(s_i) = 1) = \sum_{h \in \mathcal{H}} p(s_i, \mathcal{O}(s_i) =
1|h) \times p(h) \\
$$
Casted into log space, this becomes:
$$
\text{log } p(\mathcal{O}(s_i) = 1) = \text{log } \sum_{h \in \mathcal{H}} e^{\text{log } p(s_i,\mathcal{O}(s_i) =
1|h) + \text{ log }p(h)}
$$
Note that the right hand side of the equation is the `logsumexp` function
applied to the sum of your likelihoods and priors arrays in log space.

You task in this section is:
1. **Implement the Marginal function**. Note that the output of this function
   should still be in log space.

In [ ]:
# Your code here

## 2d. Posterior
Now we have all the necessary component to obtain the posterior. The Bayes
theorem shows that the posterior probability of a rule given the previous
observations is:
$$
\begin{align}
P(\mathcal{h}|s_i, \mathcal{O}(s_i) = 1) &= \frac{p(s_i, \mathcal{O}(s_i) =
1|h) \times p(h)}{p(s_i, \mathcal{O}(s_i) = 1)} \\
&= \frac{\text{likelihood}_h \times \text{prior}_h}{\text{Marginal}_\mathcal{H}}
\\
&= e^{\text{log likelihood}_h + \text{ log prior}_h - \text{ log Marginal}_\mathcal{H}}
\end{align}
$$
Of course, when you take away the exponent the last formula will give you a log
posterior. By replacing this posterior with the prior, you update the model with
the results according to the Bayes theorem.

Your tasks in this section are:
1. **Implement the posterior function**. Again, keep track of the space you are
   in; you do not want to update your log prior with a natural posterior.
2. **Run your Type-Singular and Counting Conjunction model through all $36$
   unique stimuli and get the top $3$ rules at the end**. That is, after
   updating the model with all $36$ unique stimuli, find the top $3$ rules with
   the highest posterior. You should be able to get the $36$ stimuli from the G1
   block of the subject's data. 

In [ ]:
# Your code here

<hr>

# Section 3. Evaluating Model Performances
Now that you have working Bayesian modela, it's time to put them into tests. The
basic idea is that you present the stimuli to the model in the same order they
are shown to the subject. You should have loaded the experiment data in
**section 1a**. Beside the `Seq` and `Blc` column, you should pay attention to
the `Rsp` and `Truth` columns, which encode the subjects' response and the
truth. `1` indicates the effect happens. Your task in this section is:
1. **Update your model with the stimuli in `Blc` `1` and `2`.**
2. **Freeze your model and obtain its prediction for the stimuli in `Blc`
   `G1` and `G2`.** That is, you should not update the prior with posterior in
   these two secionts. You should measure the models' performance in two ways (you should revert your models' prediction back to the
	nature space to do this.):
	- 2.1. **Measure the absoluate difference between Model's prediction and the
	ground truth**
	- 2.2 **Measure the absoluate difference between Model's prediction and
	subject's responses**
3. **Plot the models' performances in bar plots.**


In [ ]:
# Your code here

<hr>

# Section 4. Fitting Models to Subjects
The stock Bayesian model we have are doing Bayesian inferences, but they are too
good to approximate subjects' learning processes. Model fitting is a huge
problem in it's own, so we will only touch on the basics of basics here. Recall
that, in **section 2b.**, we $m$ as a free parameter to control the uncertainty
of the model. We will vary $m$ to obtain a better model fit. In addition, we
will introduce two extra free parameters to further fit the subjects. Recall
that, in **section 2c.** we defined the marginal as:
$$
p(\mathcal{O}(s_i) = 1) = \sum_{h \in \mathcal{H}} p(s_i, \mathcal{O}(s_i) =
1|h) \times p(h) \\
$$
we will add two extra free parameters, $\alpha, \beta$ so that: 
$$
\begin{align}
p(\mathcal{O}(s_i) = 1) = & \;\; \alpha \times [\sum_{h \in \mathcal{H}} p(s_i, \mathcal{O}(s_i) =
1|h) \times p(h)] \\
& + (1 - \alpha) \times \beta
\end{align}
$$
where $0 \leq \alpha \leq 1$ represents the subjects' tendency to make
predictions according to their own hypotheses and $0 \leq \beta \leq 1$ represents
the subjects' tendency toward responding positively. That is, in $\alpha$ of trials
subjects respond as an ideal bayesian learner, and in $(1 - \alpha)$ of trials
subjects respond randomly. Importantly, **do not use this Marginal to update the
posterior** (although I am on the fence about whether this is the correct way of
doing things, let's keep things this way for now). In each trial, you should collect two set of data: the posterior
probability obtained in **section 2c.**, and the model's fitted prediction
obtained in this section. While you use the latter to generate the model's
prediction, you use the former to do Bayesian updates.

Now we have three free parameters: $m, \alpha, \beta$. While $m$ varies between
$0.5$ and $1$, $\alpha$ and $\beta$ varies between $0$ and $1$. We are going to
fit the subject's data using grid search. We should test all possible values for
each parameters with a $0.1$ spacing. Including the two extremes, this will give
us $6$ values for $m$ and $11$ values for $\alpha$ and $\beta$. This results in
$6 \times 11 \times 11 = 726$ combinations of values for the three parameters. 

Your tasks in this section are:
1. **Test all $726$ value combinations of the three free parameter for each
   model and get the best combination for each model**. The performance is
   measured in the absolute difference between models' and subject's predictions
   introduced in **section 3**.
2. **Reproduce the performance bar plots of the best models**.

In [ ]:
# Your code here